In [19]:
#kütüphanelerin importlanması
import tensorflow as tf
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install keras

In [ ]:
#spectogram yolu oluşturulması ve spectogram fotografların yüklenmesi
path = "/content/gdrive/MyDrive/Colab Notebooks/spectrograms"
X = []
y = []
for i in range(10):
    os.chdir(f"{path}/{i}")
    for file in os.listdir():
        image = cv2.imread(f"{path}/{i}/{file}")
        image_resized = cv2.resize(image,(128, 96))
    #JPEG array yapma
        X.append(image_resized)
        y.append(i)
X = np.array(X)
y = np.array(y)
print(X[0].shape)





In [ ]:
#0'ıncı indeksi gösterelim
plt.imshow(X[0])
plt.show()

In [ ]:
#x ve y nin veri setlerini oluşturalım
X_train, X_temporary, y_train, y_temporary = train_test_split(X, y, train_size=0.8)


X_val, X_test, y_val, y_test = train_test_split(X_temporary, y_temporary, train_size=0.5)

In [ ]:
plt.imshow(X_test[669])
plt.show()

In [ ]:
#veri setindeki değişkenlerin uzunluklarını göstgerelim
print(f'veri seti uzunluğu : {len(X)}')
print(f'training veri setinin uzunluğu: {len(X_train)}')
print(f'validation veri setinin uzunluğu: {len(X_val)}')
print(f'test veri setinin uzunluğu: {len(X_test)}')

In [ ]:
#veri setinin 0-1 arası değer alması için normalizasyon (/255)uyguluyoruz

X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

In [ ]:
X_test[0]

In [ ]:
#image_width = 25
#image_height = 25

In [ ]:
images = []
for category in range(10):
    max_img_number = 0
    index = categories.index(category)
    for image in os.listdir(f"{path}/{category}"):
        if max_img_number == 1000:
            break
        img_path = f"{path}/{category}/{image}"
        try:
            img = cv2.imread(img_path,cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img,(128,128))
            img = img/255   
            images.append([index,img])
            max_img_number += 1
        except Exception as e:
            print(e)
            pass

  

##model hazırlanamsı ve eğitimi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
drive.mount('/content/gdrive', force_remount=True)


from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import adam_v2
from keras.optimizers import sgd_experimental
from keras.optimizers import rmsprop_v2
#from keras.optimizers import Adam
#from keras.optimizers import sgd_experimental
#from keras.optimizers import rmsprop_v2

In [ ]:
picture_size = 48
folder_path = "/content/drive/MyDrive/Colab Notebooks/spectrograms"



In [ ]:
no_of_classes = 10

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


In [ ]:
model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
#optimizer = adam_v2.Adam(learning_rate=lr, decay=lr/epochs)
#opt = adam_v2(lr = 0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

epochs = 48

from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Sequential,optimizers

IMG_HEIGHT = 64
IMG_WIDTH = 64

def create_model(kernel_size: tuple, input_shape: tuple):
  model = Sequential()
  model.add(Conv2D(IMG_WIDTH,
                  kernel_size=kernel_size,
                  strides=(1,1),
                  padding="same",
                  activation="relu",
                  input_shape=input_shape)           
  )
  model.add(MaxPooling2D((2,2)))
  model.add(Conv2D(IMG_WIDTH/2,
                  kernel_size=kernel_size,
                  strides=(1,1),
                  padding="same",
                  activation="relu",
                  input_shape=input_shape)           
  )

  model.add(Flatten())
  model.add(Dense(64, activation='relu')),
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=optimizers.Adam(lr=0.0001), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
  # tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  return model

In [ ]:
my_model = create_model((3, 3), (IMG_WIDTH, IMG_HEIGHT, 1))
results = my_model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data = (X_val, y_val))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(results.history["loss"], label="Loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
!pwd


In [ ]:
!ls